# Лабораторная работа №2

In [1]:
import numpy as np

### В данной работе будут реализованы численные методы решения СЛАУ, сравнены полученные результаты с точными методами, оценены погрешности, найдено число обусловленности в различных нормах, посчитаны числа неувязок, а также посчитаны максимальное и минимальное собственные числа матрицы.

## Я буду использовать матрицу и столбец из пункта л):

In [2]:
n = 20
matrix = np.zeros((n, n))
rightFunction = np.array([[1 / i] for i in range(1, n + 1)])
for i in range(n):
    for j in range(n):
        if i == j:
            matrix[i, j] = 10
        else:
            matrix[i, j] = 1 / (i + j + 2)


## Определим функции трех норм для вектора x и матрицы A:
- $||x||_1 = \max\limits_{k} |x_k|$; $||A||_1 = \max\limits_{1 \leq i \leq n} \sum\limits_{1 \leq j \leq n}^{n} |a_{ij}|$ 
- $||x||_2 = \sum\limits_{k} |x_k|$; $||A||_2 = \max\limits_{1 \leq j \leq n} \sum\limits_{1 \leq i \leq n}^{n} |a_{ij}|$ 
- $||x||_3 = \sqrt{(x, x)}$; $||A||_3 = \sqrt{|\max\limits_{1 \leq i \leq n} \lambda^{i} \cdot (A^{*} A)|}$
## А также сразу напишем способ определения масимального и минимального собственных значений матрицы

In [3]:
def vectorNorm1(vec):
    return np.abs(vec).max()

def vectorNorm2(vec):
    return np.abs(vec).sum()

def vectorNorm3(vec):
    return np.sqrt(np.dot(vec.T, vec)).item()

def matrixNorm1(A):
    return np.abs(np.abs(A).sum(axis=1, dtype='float')).max()

def matrixNorm2(A):
    return np.abs(np.abs(A).sum(axis=0, dtype='float')).max()
    
def matrixNorm3(A):
    return np.sqrt(maxEig(np.matmul(A.T, A)))

def maxEig(A, init_vec = None, numIter = 100):
    x = np.random.rand(A.shape[1]) if init_vec is None else init_vec;
    for i in range(numIter):
        x = np.dot(A, x) / vectorNorm1(x)
    
    return np.dot(np.dot(x.T, A), x) / np.dot(x.T, x)

def minEig(A, init_vec = None, numIter = 100):
    x = np.random.rand(A.shape[1]) if init_vec is None else init_vec;
    return 1 / maxEig(np.linalg.inv(A), x, numIter)

w = np.linalg.eigvals(matrix)

compMaxEig = maxEig(matrix)
trueMax = max(w)
print('Compute max eigen value: ' + str(compMaxEig))
print('True max eigen value: ' + str(trueMax))
print('Error: ' + str((compMaxEig - trueMax) / trueMax))

compMinEig = minEig(matrix)
trueMin = min(w)
print('Compute min eigen value: ' + str(compMinEig))
print('True min eigen value: ' + str(trueMin))
print('Error: ' + str((compMinEig - trueMin) / trueMin))

Compute max eigen value: 11.300128398417678
True max eigen value: 11.300128398425878
Error: -7.256256638476585e-13
Compute min eigen value: 9.712398553645631
True min eigen value: 9.657797124947598
Error: 0.005653611065922


## Как видно, собственные значения получаются довольно точно относительно библиотечных методов (при 100 итерациях погрешность погрешность для данной матрицы):
- у максимального $\approx10^{-12}$
- у минимального $\approx10^{-3}$

## Также напишем функцию для числа обусловленности

In [4]:
def conditionNumber(A, norm):
    return norm(A) * norm (np.linalg.inv(A))

print(conditionNumber(matrix, matrixNorm1))
print(conditionNumber(matrix, matrixNorm2))
print(conditionNumber(matrix, matrixNorm3))

1.4526954762585125
1.4526954762585123
1.1700373868496845


## Видно, что число обусловленности гораздо меньше 100, так что задача обусловлена хорошо

## Проверим возможность LU разложения:

In [5]:
def LU_is_available(A):
    for i in range(A.shape[0]):
        if np.linalg.det(A[:i, :i]) == 0:
            return False
    return True

LU_is_available(matrix)

True

## Так как разложение возможно, то выполним его:

In [6]:
def LU (A):
    n = A.shape[0]
    L = np.eye(n)
    U = np.zeros((n, n))
    
    for i in range(n):
        for j in range(n):
            if i <= j:
                U[i][j] = A[i][j] - sum(np.dot(L[i:i+1, 0:i], U[0:i, j:j+1]))
            else:
                L[i][j] = (A[i][j] - sum(np.dot(L[i:i+1, 0:j], U[0:j, j:j+1]))) / U[j][j]
    return L, U

def get_l_solution(L, f):
    y = np.zeros((L.shape[0], 1))
    assert L[0][0] != 0
    y[0] = f[0] / L[0][0]
    for i in range(1, len(y)):
        assert L[i][i] != 0
        y[i] = (f[i] - sum(np.dot(L[i:i+1, :i], y[:i]))) / L[i][i]
    return y

def get_u_solution(U, y):
    x = np.zeros((U.shape[0], 1))
    assert U[-1][-1] != 0
    
    x[-1] = y[-1] / U[-1][-1]
    for i in range(U.shape[0] - 1, -1, -1):
        x[i] = (y[i] - sum(np.dot(U[i:i+1, i+1:], x[i+1:]))) / U[i][i]
    return x

L, U = LU(matrix)
y = get_l_solution(L, rightFunction)
ans = get_u_solution(U, y)
print("My solution:")
print(ans)

My solution:
[[0.09615746]
 [0.04478375]
 [0.02874755]
 [0.02101576]
 [0.0164943 ]
 [0.01353947]
 [0.01146294]
 [0.00992677]
 [0.00874602]
 [0.00781115]
 [0.00705327]
 [0.00642689]
 [0.0059008 ]
 [0.00545292]
 [0.00506717]
 [0.00473155]
 [0.00443698]
 [0.00417641]
 [0.00394434]
 [0.00373636]]


## Сделаем печать невязок

In [7]:
print("Errors:")
errors = np.matmul(matrix, ans) - rightFunction
print(f"Norm of error vector in 1st norm = {vectorNorm1(errors)}")
print(f"Norm of error vector in 2nd norm = {vectorNorm2(errors)}")
print(f"Norm of error vector in 3rd norm = {vectorNorm3(errors)}")

Errors:
Norm of error vector in 1st norm = 1.1102230246251565e-16
Norm of error vector in 2nd norm = 2.7755575615628914e-16
Norm of error vector in 3rd norm = 1.4015878649856313e-16


## Теперь решим методом верхней релаксации

In [8]:
def upperRelaxation_is_available(A):
    for i in range(A.shape[0]):
        if np.linalg.det(A[:i, :i]) <= 0:
            return False
    return True

upperRelaxation_is_available(matrix)

True

## Как видно, метод применим. Критерием останова посчитаем $||f - A \cdot x||_3 < \varepsilon$, при $\varepsilon = 10^{-14}$

In [9]:
def upperRelaxation(A, f, w=0.1, epsilon=1e-14, init_vec = None, norm=vectorNorm3):
    n = A.shape[0]
    x = np.random.rand(1, n) if init_vec is None else init_vec;
    x = x.T
    
    D = np.zeros([n, n])
    L = np.zeros([n, n])
    U = np.zeros([n, n])
    
    for i in range(0, n):
        for j in range(0, n):
            if i > j:
                L[i, j] = A[i, j]
            elif i < j:
                U[i, j] = A[i, j]
            else:
                D[i, j] = A[i, j]
    
    first = np.linalg.inv(D + w * L)
    second = np.matmul(first, (w - 1) * D + w * U)
    
    numIter = 0
    
    while norm(f - np.matmul(A, x)) >= epsilon:
        prev = x
        x = -np.matmul(second, x) + np.matmul(w * first, f)
        numIter += 1

    return (x, numIter)

solution, numIter = upperRelaxation(matrix, rightFunction)
print("Number of iterations: " + str(numIter))
print("My solution:")
print(solution)

Number of iterations: 336
My solution:
[[0.09615746]
 [0.04478375]
 [0.02874755]
 [0.02101576]
 [0.0164943 ]
 [0.01353947]
 [0.01146294]
 [0.00992677]
 [0.00874602]
 [0.00781115]
 [0.00705327]
 [0.00642689]
 [0.0059008 ]
 [0.00545292]
 [0.00506717]
 [0.00473155]
 [0.00443698]
 [0.00417641]
 [0.00394434]
 [0.00373636]]


## Собственно видно, что при выставлении достаточной точности результаты получаются не хуже точного метода из-за машинной погрешности и за 336 итераций получено приемлемое решение. Сделаем печать невязок:

In [10]:
print("Errors:")
errors = np.matmul(matrix, solution) - rightFunction
print(f"Norm of error vector in 1st norm = {vectorNorm1(errors)}")
print(f"Norm of error vector in 2nd norm = {vectorNorm2(errors)}")
print(f"Norm of error vector in 3rd norm = {vectorNorm3(errors)}")

Errors:
Norm of error vector in 1st norm = 2.942091015256665e-15
Norm of error vector in 2nd norm = 3.722022690055837e-14
Norm of error vector in 3rd norm = 9.074338337022835e-15
